In [1]:
import os
from config_file import config
import sys
sys.path.append("/export/home/nidebroux/sct_custom")
sys.path.append("/export/home/nidebroux/modules")
os.environ["CUDA_VISIBLE_DEVICES"] = '/gpu:0'
import numpy as np
import pandas as pd
import pickle
import random
from spinalcordtoolbox.image import Image
from sklearn.utils import shuffle
import tkinter
import matplotlib
#comment the line below if necessary
matplotlib.use('TkAgg')
import keras
import tensorflow as tf
from spinalcordtoolbox.deepseg_sc.cnn_models_3d import load_trained_model
from generator import get_training_and_validation_generators
from utils import fetch_data_files, visualize_data, normalize_data, load_3Dpatches, train_model, get_callbacks
from keras import backend as K
import matplotlib.pyplot as plt

main_dir = config["main_dir"]

### IMPORT PARAMETERS FROM CONFIG FILE

In [2]:
# config['data_dict'] = pickle file containing a dictionary with at least the following keys: subject and contrast_foldname
# This dict is load as a panda dataframe and used by the function utils.fetch_data_files
# IMPORTANT NOTE: the testing dataset is not included in this dataframe
DATA_PD = pd.read_pickle(config['data_dict'])

DATA_FOLD = config["data_dir"]  # where the preprocess data are stored
#MODEL_FOLD = config["path2save"]  # where to store the trained models

MEAN_TRAIN_T2, STD_TRAIN_T2 = 871.309, 557.916  # Mean and SD of the training dataset of the original paper

### CONVERT INPUT IMAGES INTO AN HDF5 FILE

In [3]:
len_train = int(0.8 * len(DATA_PD.index)) # 80% of the dataset is used for the training, 20% for validation
idx_train = random.sample(range(len(DATA_PD.index)), len_train)
idx_valid = [ii for ii in range(len(DATA_PD.index)) if ii not in idx_train] # the remaining images are used for the validation

In [4]:
training_files = fetch_data_files(data_frame=DATA_PD[DATA_PD.index.isin(idx_train)],
                                  data_fold=DATA_FOLD,
                                  im_suffixe='_norm',
                                  target_suffixe='_crop_SEG')
validation_files = fetch_data_files(data_frame=DATA_PD[DATA_PD.index.isin(idx_valid)],
                                  data_fold=DATA_FOLD,
                                  im_suffixe='_norm',
                                  target_suffixe='_crop_SEG')

print(training_files)
print(validation_files)

[('/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz', '/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_crop_SEG.nii.gz'), ('/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz', '/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_crop_SEG.nii.gz'), ('/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz', '/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_crop_SEG.nii.gz'), ('/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz', '/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_crop_SEG.nii.gz'), ('/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz', '/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_crop_SEG.nii.gz'), ('/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz', '/home/nidebroux/data_mask/preprocess/Data_organize

### EXTRACT 3D PATCHES

In [6]:
# The extracted patches are stored as pickle files (one for training, one for validation).
# If these files already exist, we load them directly (i.e. do not re run the patch extraction).
pkl_train_fname = DATA_FOLD + 'train_data.pkl'
print(pkl_train_fname)
if not os.path.isfile(pkl_train_fname):
    
    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=config["patch_overlap"]) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)
    
    with open(pkl_train_fname, 'wb') as fp:
        pickle.dump(np.array([X_train, y_train]), fp)
else:
    with open (pkl_train_fname, 'rb') as fp:
        X_train, y_train = pickle.load(fp)

/home/nidebroux/data_mask/preprocess/Data_organized/train_data.pkl


In [6]:

pkl_valid_fname = DATA_FOLD + 'valid_data.pkl'
print(pkl_valid_fname)

if not os.path.isfile(pkl_valid_fname):
    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    with open(pkl_valid_fname, 'wb') as fp:
        pickle.dump(np.array([X_valid, y_valid]), fp)
else:
    with open (pkl_valid_fname, 'rb') as fp:
        X_valid, y_valid = pickle.load(fp)

/home/nidebroux/data_mask/preprocess/Data_organized/valid_data.pkl


In [7]:
print('Number of Training patches:\n\t' + str(X_train.shape[0]))
print('Number of Validation patches:\n\t' + str(X_valid.shape[0]))

Number of Training patches:
	271
Number of Validation patches:
	38


### LOAD TRAINED MODEL

In [5]:
model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')
loss =model.loss
optimizer_class_name = model.optimizer.__class__.__name__
optimizer_config = model.optimizer.get_config()
model.compile(optimizer = optimizer_class_name, loss=loss)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### GET TRAINING AND VALIDATION GENERATORS

In [9]:
train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=config["batch_size"],
                                                    augment=True,
                                                    augment_flip=True)

print(train_generator,nb_train_steps)

<generator object data_generator at 0x7fdd08e817d0> 271


In [10]:
validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=1,
                                                    augment=False,
                                                    augment_flip=False)
print(validation_generator,nb_valid_steps)

<generator object data_generator at 0x7fdcf4124250> 38


### MAIN FINE-TUNING PIPELINE

The fine-tuning technique and parameters chosen here are based on the results obtained trough the tests below. It depends directly from the training set and should thus be change in function of the used training set.

In [ ]:
K.set_value(model.optimizer.learning_rate,0.00001)

history = train_model(model=model,
    path2save=config["path2save"],
    model_name=config["model_name"],
    training_generator=train_generator,
    validation_generator=validation_generator,
    steps_per_epoch=nb_train_steps,
    validation_steps=nb_valid_steps,
    n_epochs=2*config["n_epochs_1"],
    learning_rate_drop=config["learning_rate_drop"],
    learning_rate_patience=config["learning_rate_patience"]
   )

np.save(main_dir + "parameters_analysis/best_results.npy",history.history['val_loss'])

# PARAMETER TESTS

The rest of the script contains a serie of tests made on the following parameters : depth of the network surgery in case of transfer learning, amount of frozen parameters during unfreezing phase, batch size, overlap size, learning rate (for transfer learning and for unfreezing phase) and data augmentation.

### NETWORK SURGERY 


We test here what is the best amount of layers to retrain from scratch.

In [13]:
#surgery from conv3d_5

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

for layer in long_model.layers[:-9]:
    layer.trainable = False
    
long_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(long_model.optimizer.learning_rate,0.001)


In [14]:
#surgery from conv3d_6

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
x_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'new_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-6].output)
x_2 = keras.layers.BatchNormalization(name = 'new_batch_1')(x_1)
x_3 = keras.layers.Activation('relu',name = 'new_activation_1')(x_2)
x_4 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'new_conv3D_2',kernel_initializer = init)(x_3)
new_output = keras.layers.Activation('sigmoid',name = 'new_activation_2')(x_4)

middle_model = keras.models.Model(model.input,new_output,name = "middle_surgery_model")

for layer in middle_model.layers[:-6]:
    layer.trainable = False

    
middle_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(middle_model.optimizer.learning_rate,0.001)

In [15]:
#surgery from conv3d_7

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
short_1 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'short_conv3D',kernel_initializer = init)(model.layers[-3].output)
short_output = keras.layers.Activation('sigmoid',name = 'short_activation')(short_1)

short_model = keras.models.Model(model.input,short_output,name = "short_surgery_model")

for layer in short_model.layers[:-3]:
    layer.trainable = False

    
short_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(short_model.optimizer.learning_rate,0.001)

In [16]:
#fit the long model
long_history = train_model(model=long_model,
            path2save=config["path2save_transferlearning"],
            model_name="long_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )
np.save(main_dir + "parameters_analysis/long_surgery.npy",long_history.history['val_loss'])

Epoch 1/100
271/271 - 17s - loss: -2.5253e-01 - val_loss: -3.8458e-01 - lr: 0.0010
Epoch 2/100
271/271 - 16s - loss: -3.7342e-01 - val_loss: -4.4275e-01 - lr: 0.0010
Epoch 3/100
271/271 - 16s - loss: -3.9122e-01 - val_loss: -3.5356e-01 - lr: 0.0010
Epoch 4/100
271/271 - 17s - loss: -4.2631e-01 - val_loss: -5.0070e-01 - lr: 0.0010
Epoch 5/100
271/271 - 17s - loss: -4.4790e-01 - val_loss: -4.0220e-01 - lr: 0.0010
Epoch 6/100
271/271 - 17s - loss: -4.4723e-01 - val_loss: -4.8898e-01 - lr: 0.0010
Epoch 7/100
271/271 - 17s - loss: -4.6730e-01 - val_loss: -4.9850e-01 - lr: 0.0010
Epoch 8/100
271/271 - 17s - loss: -4.7749e-01 - val_loss: -5.1019e-01 - lr: 0.0010
Epoch 9/100
271/271 - 17s - loss: -4.8746e-01 - val_loss: -3.8670e-01 - lr: 0.0010
Epoch 10/100
271/271 - 17s - loss: -5.0616e-01 - val_loss: -5.0649e-01 - lr: 0.0010
Epoch 11/100
271/271 - 17s - loss: -5.0503e-01 - val_loss: -4.9474e-01 - lr: 0.0010
Epoch 12/100
271/271 - 17s - loss: -5.1997e-01 - val_loss: -4.9004e-01 - lr: 0.0010
E

Epoch 92/100
271/271 - 17s - loss: -6.3844e-01 - val_loss: -6.3214e-01 - lr: 3.1250e-05
Epoch 93/100

Epoch 00093: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
271/271 - 17s - loss: -6.3634e-01 - val_loss: -6.1163e-01 - lr: 3.1250e-05
Epoch 94/100
271/271 - 17s - loss: -6.3727e-01 - val_loss: -5.6424e-01 - lr: 1.5625e-05
Epoch 95/100
271/271 - 17s - loss: -6.3579e-01 - val_loss: -6.5033e-01 - lr: 1.5625e-05
Epoch 96/100
271/271 - 17s - loss: -6.3708e-01 - val_loss: -6.2647e-01 - lr: 1.5625e-05
Epoch 97/100
271/271 - 17s - loss: -6.3752e-01 - val_loss: -5.8697e-01 - lr: 1.5625e-05
Epoch 98/100
271/271 - 17s - loss: -6.3850e-01 - val_loss: -6.1642e-01 - lr: 1.5625e-05
Epoch 99/100
271/271 - 17s - loss: -6.3877e-01 - val_loss: -5.7751e-01 - lr: 1.5625e-05
Epoch 100/100
271/271 - 17s - loss: -6.3987e-01 - val_loss: -6.2354e-01 - lr: 1.5625e-05


In [17]:
#fit the middle model
middle_history = train_model(model=middle_model,
            path2save=config["path2save_transferlearning"],
            model_name="middle_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )
np.save(main_dir + "parameters_analysis/middle_surgery.npy",middle_history.history['val_loss'])

Epoch 1/100
271/271 - 10s - loss: -2.2929e-01 - val_loss: -2.7286e-01 - lr: 0.0010
Epoch 2/100
271/271 - 10s - loss: -2.9437e-01 - val_loss: -3.3093e-01 - lr: 0.0010
Epoch 3/100
271/271 - 9s - loss: -3.0390e-01 - val_loss: -3.5035e-01 - lr: 0.0010
Epoch 4/100
271/271 - 9s - loss: -3.2746e-01 - val_loss: -3.3819e-01 - lr: 0.0010
Epoch 5/100
271/271 - 10s - loss: -3.2941e-01 - val_loss: -4.1459e-01 - lr: 0.0010
Epoch 6/100
271/271 - 9s - loss: -3.4385e-01 - val_loss: -3.3247e-01 - lr: 0.0010
Epoch 7/100
271/271 - 9s - loss: -3.4003e-01 - val_loss: -3.5968e-01 - lr: 0.0010
Epoch 8/100
271/271 - 10s - loss: -3.4934e-01 - val_loss: -4.2398e-01 - lr: 0.0010
Epoch 9/100
271/271 - 9s - loss: -3.4986e-01 - val_loss: -3.7006e-01 - lr: 0.0010
Epoch 10/100
271/271 - 9s - loss: -3.6116e-01 - val_loss: -3.9322e-01 - lr: 0.0010
Epoch 11/100
271/271 - 9s - loss: -3.6051e-01 - val_loss: -3.3979e-01 - lr: 0.0010
Epoch 12/100
271/271 - 9s - loss: -3.6198e-01 - val_loss: -3.6012e-01 - lr: 0.0010
Epoch 13/

Epoch 90/100
271/271 - 9s - loss: -4.3938e-01 - val_loss: -4.1610e-01 - lr: 7.8125e-06
Epoch 91/100
271/271 - 9s - loss: -4.3838e-01 - val_loss: -4.2144e-01 - lr: 7.8125e-06
Epoch 92/100

Epoch 00092: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
271/271 - 9s - loss: -4.4005e-01 - val_loss: -4.2899e-01 - lr: 7.8125e-06
Epoch 93/100
271/271 - 9s - loss: -4.4105e-01 - val_loss: -4.1375e-01 - lr: 3.9063e-06
Epoch 94/100
271/271 - 9s - loss: -4.4044e-01 - val_loss: -4.0720e-01 - lr: 3.9063e-06
Epoch 95/100
271/271 - 9s - loss: -4.4047e-01 - val_loss: -4.4945e-01 - lr: 3.9063e-06
Epoch 96/100
271/271 - 9s - loss: -4.4206e-01 - val_loss: -3.9712e-01 - lr: 3.9063e-06
Epoch 97/100
271/271 - 9s - loss: -4.3891e-01 - val_loss: -4.4289e-01 - lr: 3.9063e-06
Epoch 98/100
271/271 - 9s - loss: -4.3660e-01 - val_loss: -4.1761e-01 - lr: 3.9063e-06
Epoch 99/100
271/271 - 9s - loss: -4.3736e-01 - val_loss: -3.8341e-01 - lr: 3.9063e-06
Epoch 100/100
271/271 - 9s - loss: -4.3957e-01 - 

In [18]:
#fit the short model
short_history = train_model(model=short_model,
            path2save=config["path2save_transferlearning"],
            model_name="short_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=config["learning_rate_patience"]
           )
np.save(main_dir + "parameters_analysis/short_surgery.npy",short_history.history['val_loss'])

Epoch 1/100
271/271 - 7s - loss: -1.0338e-01 - val_loss: -9.9860e-02 - lr: 0.0010
Epoch 2/100
271/271 - 6s - loss: -1.2411e-01 - val_loss: -1.1583e-01 - lr: 0.0010
Epoch 3/100
271/271 - 6s - loss: -1.3932e-01 - val_loss: -1.2746e-01 - lr: 0.0010
Epoch 4/100
271/271 - 6s - loss: -1.5197e-01 - val_loss: -1.4523e-01 - lr: 0.0010
Epoch 5/100
271/271 - 6s - loss: -1.5819e-01 - val_loss: -1.5943e-01 - lr: 0.0010
Epoch 6/100
271/271 - 6s - loss: -1.5808e-01 - val_loss: -1.3233e-01 - lr: 0.0010
Epoch 7/100
271/271 - 6s - loss: -1.6588e-01 - val_loss: -1.7146e-01 - lr: 0.0010
Epoch 8/100
271/271 - 6s - loss: -1.6767e-01 - val_loss: -1.6437e-01 - lr: 0.0010
Epoch 9/100
271/271 - 6s - loss: -1.6967e-01 - val_loss: -1.5643e-01 - lr: 0.0010
Epoch 10/100
271/271 - 6s - loss: -1.7543e-01 - val_loss: -1.7140e-01 - lr: 0.0010
Epoch 11/100
271/271 - 6s - loss: -1.7362e-01 - val_loss: -1.7003e-01 - lr: 0.0010
Epoch 12/100
271/271 - 6s - loss: -1.7699e-01 - val_loss: -1.7521e-01 - lr: 0.0010
Epoch 13/100


Epoch 93/100
271/271 - 6s - loss: -2.0357e-01 - val_loss: -1.9089e-01 - lr: 6.2500e-05
Epoch 94/100
271/271 - 6s - loss: -2.0495e-01 - val_loss: -2.0460e-01 - lr: 6.2500e-05
Epoch 95/100
271/271 - 6s - loss: -2.0317e-01 - val_loss: -1.9311e-01 - lr: 6.2500e-05
Epoch 96/100

Epoch 00096: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
271/271 - 6s - loss: -2.0316e-01 - val_loss: -1.9134e-01 - lr: 6.2500e-05
Epoch 97/100
271/271 - 6s - loss: -2.0390e-01 - val_loss: -2.0176e-01 - lr: 3.1250e-05
Epoch 98/100
271/271 - 6s - loss: -2.0097e-01 - val_loss: -1.8474e-01 - lr: 3.1250e-05
Epoch 99/100
271/271 - 6s - loss: -2.0230e-01 - val_loss: -2.0468e-01 - lr: 3.1250e-05
Epoch 100/100
271/271 - 6s - loss: -2.0375e-01 - val_loss: -1.9122e-01 - lr: 3.1250e-05


In [27]:
#clone of the model without training
for layer in middle_model.layers:
    layer.trainable = True

clone_model = tf.keras.models.clone_model(model)

   
clone_model.compile(optimizer = optimizer_class_name, loss=loss)
    
K.set_value(clone_model.optimizer.learning_rate,0.001)

In [28]:
clone_history = train_model(model=clone_model,
            path2save=config["path2save_transferlearning"],
            model_name="total_surgery",
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=config["learning_rate_patience"]
           )
np.save(main_dir + "parameters_analysis/clone.npy",clone_history.history['val_loss'])

Epoch 1/100
271/271 - 21s - loss: -3.5441e-01 - val_loss: -1.4051e-01 - lr: 0.0010
Epoch 2/100
271/271 - 21s - loss: -4.8608e-01 - val_loss: -1.8485e-02 - lr: 0.0010
Epoch 3/100
271/271 - 22s - loss: -5.6707e-01 - val_loss: -2.1636e-01 - lr: 0.0010
Epoch 4/100
271/271 - 22s - loss: -5.9354e-01 - val_loss: -3.5034e-01 - lr: 0.0010
Epoch 5/100
271/271 - 22s - loss: -6.0958e-01 - val_loss: -4.0237e-02 - lr: 0.0010
Epoch 6/100
271/271 - 22s - loss: -6.1179e-01 - val_loss: -1.8814e-01 - lr: 0.0010
Epoch 7/100
271/271 - 22s - loss: -6.1240e-01 - val_loss: -2.4260e-01 - lr: 0.0010
Epoch 8/100
271/271 - 22s - loss: -6.3322e-01 - val_loss: -2.6582e-01 - lr: 0.0010
Epoch 9/100
271/271 - 22s - loss: -6.3374e-01 - val_loss: -3.5987e-01 - lr: 0.0010
Epoch 10/100
271/271 - 22s - loss: -6.4054e-01 - val_loss: -1.1434e-01 - lr: 0.0010
Epoch 11/100
271/271 - 22s - loss: -6.4976e-01 - val_loss: -1.0557e-02 - lr: 0.0010
Epoch 12/100
271/271 - 22s - loss: -6.5705e-01 - val_loss: -5.2792e-01 - lr: 0.0010
E

Epoch 92/100
271/271 - 22s - loss: -8.0374e-01 - val_loss: -6.1535e-01 - lr: 6.2500e-05
Epoch 93/100
271/271 - 22s - loss: -8.0143e-01 - val_loss: -6.8996e-01 - lr: 6.2500e-05
Epoch 94/100
271/271 - 22s - loss: -8.0299e-01 - val_loss: -5.6547e-01 - lr: 6.2500e-05
Epoch 95/100
271/271 - 22s - loss: -8.0057e-01 - val_loss: -6.7116e-01 - lr: 6.2500e-05
Epoch 96/100
271/271 - 22s - loss: -8.0243e-01 - val_loss: -5.7743e-01 - lr: 6.2500e-05
Epoch 97/100
271/271 - 22s - loss: -8.0300e-01 - val_loss: -6.0879e-01 - lr: 6.2500e-05
Epoch 98/100
271/271 - 22s - loss: -8.0439e-01 - val_loss: -6.1190e-01 - lr: 6.2500e-05
Epoch 99/100
271/271 - 22s - loss: -8.0450e-01 - val_loss: -6.0249e-01 - lr: 6.2500e-05
Epoch 100/100
271/271 - 22s - loss: -8.0371e-01 - val_loss: -6.7637e-01 - lr: 6.2500e-05


In [32]:
long_surgery = np.load(main_dir+"parameters_analysis/long_surgery.npy")
middle_surgery = np.load(main_dir+"parameters_analysis/middle_surgery.npy")
short_surgery = np.load(main_dir+"parameters_analysis/short_surgery.npy")
clone = np.load(main_dir+"parameters_analysis/clone.npy")

epochs = range(1,101,4)
mean_val_loss = np.zeros((4,25))

for i in range(25):
    mean_val_loss[0,i]=np.abs(np.mean(long_surgery[i:i+4])) 
    mean_val_loss[1,i]=np.abs(np.mean(middle_surgery[i:i+4]))
    mean_val_loss[2,i]=np.abs(np.mean(short_surgery[i:i+4]))
    mean_val_loss[3,i]=np.abs(np.mean(clone[i:i+4]))


plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'val_loss for big surgery')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'val_loss for middle surgery')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'val_loss for short surgery')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'val_loss for total replacement')
plt.title('Comparison between different surgeries ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 4 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### BATCH SIZE TEST

In [7]:
batch_size_train = [1,5,10,20,50]
val_loss_values = np.zeros((5,50))

In [8]:

for i in range(5):

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=batch_size_train[i],
                                                    augment=True,
                                                    augment_flip=True)


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=1,
                                                    augment=False,
                                                    augment_flip=False)

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
    long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
    long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
    long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
    long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
    long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
    long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
    long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

    long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

    for layer in long_model.layers[:-9]:
        layer.trainable = False

    long_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(long_model.optimizer.learning_rate,0.001)

    #fit the long model
    history = train_model(model=long_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["batch_size_test",str(nb_train_steps)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=50,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_values[i,:] = history.history['val_loss']
    
np.save(main_dir + "parameters_analysis/batch_size.npy",val_loss_values)

NameError: name 'X_train' is not defined

In [ ]:
epochs = range(5,51,5)
val_loss_values = np.load(main_dir + "parameters_analysis/batch_size.npy")
mean_val_loss = np.zeros((5,10))
for i in range(10):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_values[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_values[1,i:i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_values[2,i:i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_values[3,i:i+5]))
    mean_val_loss[4,i]=np.abs(np.mean(val_loss_values[4,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'batch size = 1 - iteration = 271')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'batch size = 5 - iteration = 54')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'batch size = 10 - iteration = 27')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'batch size = 20 - iteration = 13')
plt.plot(epochs,mean_val_loss[4,:],'o--',label = 'batch size = 50 - iteration = 5')
plt.title('Evolution of the validation loss with different batch sizes ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### OVERLAP TEST

In [8]:
overlap_array = [None,42,36,24,12]
val_loss_overlap = np.zeros((5,50))

In [9]:
for i in range(5):

    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=overlap_array[i]) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)
    
    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=5,
                                                    augment=True,
                                                    augment_flip=True)


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=2,
                                                    augment=False,
                                                    augment_flip=False)

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
    long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
    long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
    long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
    long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
    long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
    long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
    long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

    long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

    for layer in long_model.layers[:-9]:
        layer.trainable = False

    long_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(long_model.optimizer.learning_rate,0.001)

    #fit the long model
    history = train_model(model=long_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["overlap_test",str(i)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=50,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_overlap[i,:] = history.history['val_loss']
    
np.save(main_dir + "parameters_analysis/overlap.npy",val_loss_overlap)

/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-10/t2/pm-10_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-13/t2/pm-13_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-6/t2/iv-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-3/t2/iv-3_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-14/t2/pm-14_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-9/t2/iv-9_t2_norm.nii.gz
/home/nidebroux/data_mask/prepro

Epoch 13/50
31/31 - 7s - loss: -4.7203e-01 - val_loss: -4.7033e-01 - lr: 0.0010
Epoch 14/50
31/31 - 7s - loss: -4.6753e-01 - val_loss: -4.8656e-01 - lr: 0.0010
Epoch 15/50
31/31 - 7s - loss: -4.8277e-01 - val_loss: -4.8621e-01 - lr: 0.0010
Epoch 16/50
31/31 - 7s - loss: -4.7478e-01 - val_loss: -5.3991e-01 - lr: 0.0010
Epoch 17/50
31/31 - 7s - loss: -4.9439e-01 - val_loss: -4.7636e-01 - lr: 0.0010
Epoch 18/50
31/31 - 7s - loss: -4.8476e-01 - val_loss: -4.7169e-01 - lr: 0.0010
Epoch 19/50
31/31 - 7s - loss: -5.0020e-01 - val_loss: -4.5982e-01 - lr: 0.0010
Epoch 20/50
31/31 - 7s - loss: -4.9027e-01 - val_loss: -4.5931e-01 - lr: 0.0010
Epoch 21/50
31/31 - 7s - loss: -4.8913e-01 - val_loss: -4.8662e-01 - lr: 0.0010
Epoch 22/50
31/31 - 7s - loss: -4.9457e-01 - val_loss: -4.7242e-01 - lr: 0.0010
Epoch 23/50
31/31 - 7s - loss: -4.8149e-01 - val_loss: -4.6659e-01 - lr: 0.0010
Epoch 24/50
31/31 - 7s - loss: -4.9082e-01 - val_loss: -4.8824e-01 - lr: 0.0010
Epoch 25/50
31/31 - 7s - loss: -5.1387e-

Epoch 41/50
36/36 - 8s - loss: -5.7023e-01 - val_loss: -5.0437e-01 - lr: 2.5000e-04
Epoch 42/50
36/36 - 8s - loss: -5.6454e-01 - val_loss: -5.0802e-01 - lr: 2.5000e-04
Epoch 43/50
36/36 - 8s - loss: -5.7006e-01 - val_loss: -5.8131e-01 - lr: 2.5000e-04
Epoch 44/50
36/36 - 8s - loss: -5.6806e-01 - val_loss: -5.0328e-01 - lr: 2.5000e-04
Epoch 45/50
36/36 - 8s - loss: -5.7190e-01 - val_loss: -4.9180e-01 - lr: 2.5000e-04
Epoch 46/50
36/36 - 8s - loss: -5.5721e-01 - val_loss: -5.3165e-01 - lr: 2.5000e-04
Epoch 47/50
36/36 - 8s - loss: -5.6867e-01 - val_loss: -5.5470e-01 - lr: 2.5000e-04
Epoch 48/50
36/36 - 8s - loss: -5.7462e-01 - val_loss: -5.5378e-01 - lr: 2.5000e-04
Epoch 49/50
36/36 - 8s - loss: -5.7526e-01 - val_loss: -5.2636e-01 - lr: 2.5000e-04
Epoch 50/50
36/36 - 8s - loss: -5.7842e-01 - val_loss: -5.7188e-01 - lr: 2.5000e-04
/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz


Epoch 3/50
106/106 - 22s - loss: -4.3785e-01 - val_loss: -4.8567e-01 - lr: 0.0010
Epoch 4/50
106/106 - 22s - loss: -4.7056e-01 - val_loss: -4.7201e-01 - lr: 0.0010
Epoch 5/50
106/106 - 22s - loss: -4.9761e-01 - val_loss: -4.9519e-01 - lr: 0.0010
Epoch 6/50
106/106 - 22s - loss: -5.0940e-01 - val_loss: -5.1542e-01 - lr: 0.0010
Epoch 7/50
106/106 - 22s - loss: -5.0399e-01 - val_loss: -4.9883e-01 - lr: 0.0010
Epoch 8/50
106/106 - 22s - loss: -5.2284e-01 - val_loss: -5.2324e-01 - lr: 0.0010
Epoch 9/50
106/106 - 22s - loss: -5.1248e-01 - val_loss: -5.0754e-01 - lr: 0.0010
Epoch 10/50
106/106 - 22s - loss: -5.2795e-01 - val_loss: -4.6839e-01 - lr: 0.0010
Epoch 11/50
106/106 - 22s - loss: -5.2910e-01 - val_loss: -4.8304e-01 - lr: 0.0010
Epoch 12/50
106/106 - 22s - loss: -5.3713e-01 - val_loss: -5.6546e-01 - lr: 0.0010
Epoch 13/50
106/106 - 22s - loss: -5.4820e-01 - val_loss: -4.5903e-01 - lr: 0.0010
Epoch 14/50
106/106 - 22s - loss: -5.5197e-01 - val_loss: -3.5909e-01 - lr: 0.0010
Epoch 15/50

In [7]:
val_loss_overlap = np.load(main_dir+"parameters_analysis/overlap.npy")
epochs = range(5,51,5)
mean_val_loss = np.zeros((5,10))
for i in range(10):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_overlap[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_overlap[1,i:i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_overlap[2,i:i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_overlap[3,i:i+5]))
    mean_val_loss[4,i]=np.abs(np.mean(val_loss_overlap[4,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'No overlap (= 48)')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'overlap of 42')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'overlap of 36')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'overlap of 24')
plt.plot(epochs,mean_val_loss[4,:],'o--',label = 'overlap of 12')
plt.title('Evolution of the validation loss with different overlap sizes ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### LEARNING RATE TEST

In [7]:
learning_array = [0.1,0.01,0.001,0.0001]
val_loss_learning = np.zeros((4,100))

In [8]:
X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=12) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)


for i in range(4):



    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
    long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
    long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
    long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
    long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
    long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
    long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
    long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

    long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

    for layer in long_model.layers[:-9]:
        layer.trainable = False

    long_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(long_model.optimizer.learning_rate,learning_array[i])

    #fit the long model
    history = train_model(model=long_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["learning_rate_test",str(learning_array[i])]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=100,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_learning[i,:] = history.history['val_loss']
    
np.save(main_dir+"parameters_analysis/learning_rate.npy",val_loss_learning)

/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-10/t2/pm-10_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-1/t2/pm-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-3/t2/iv-3_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-14/t2/pm-14_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-9/t2/iv-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-6/t2/pm-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preproce

Epoch 78/100
106/106 - 22s - loss: -6.0991e-01 - val_loss: -6.1299e-01 - lr: 0.0125
Epoch 79/100
106/106 - 22s - loss: -6.0704e-01 - val_loss: -6.4674e-01 - lr: 0.0125
Epoch 80/100
106/106 - 22s - loss: -6.0527e-01 - val_loss: -6.1678e-01 - lr: 0.0125
Epoch 81/100
106/106 - 22s - loss: -6.0653e-01 - val_loss: -6.7362e-01 - lr: 0.0125
Epoch 82/100
106/106 - 22s - loss: -6.0937e-01 - val_loss: -6.9708e-01 - lr: 0.0125
Epoch 83/100
106/106 - 22s - loss: -6.0795e-01 - val_loss: -6.4699e-01 - lr: 0.0125
Epoch 84/100
106/106 - 22s - loss: -6.0917e-01 - val_loss: -6.1964e-01 - lr: 0.0125
Epoch 85/100
106/106 - 22s - loss: -6.1005e-01 - val_loss: -6.3073e-01 - lr: 0.0125
Epoch 86/100
106/106 - 22s - loss: -6.0761e-01 - val_loss: -6.3345e-01 - lr: 0.0125
Epoch 87/100
106/106 - 22s - loss: -6.1034e-01 - val_loss: -6.3456e-01 - lr: 0.0125
Epoch 88/100
106/106 - 22s - loss: -6.1523e-01 - val_loss: -5.9279e-01 - lr: 0.0125
Epoch 89/100
106/106 - 22s - loss: -6.0745e-01 - val_loss: -6.7046e-01 - lr:

Epoch 72/100
106/106 - 22s - loss: -6.1610e-01 - val_loss: -6.5540e-01 - lr: 0.0012
Epoch 73/100
106/106 - 22s - loss: -6.2050e-01 - val_loss: -5.0968e-01 - lr: 0.0012
Epoch 74/100
106/106 - 22s - loss: -6.2171e-01 - val_loss: -6.5452e-01 - lr: 0.0012
Epoch 75/100
106/106 - 22s - loss: -6.1896e-01 - val_loss: -6.6029e-01 - lr: 0.0012
Epoch 76/100
106/106 - 22s - loss: -6.2075e-01 - val_loss: -6.3658e-01 - lr: 0.0012
Epoch 77/100
106/106 - 22s - loss: -6.2102e-01 - val_loss: -5.9845e-01 - lr: 0.0012
Epoch 78/100

Epoch 00078: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
106/106 - 22s - loss: -6.2121e-01 - val_loss: -6.7748e-01 - lr: 0.0012
Epoch 79/100
106/106 - 22s - loss: -6.2314e-01 - val_loss: -5.8916e-01 - lr: 6.2500e-04
Epoch 80/100
106/106 - 22s - loss: -6.2097e-01 - val_loss: -6.4477e-01 - lr: 6.2500e-04
Epoch 81/100
106/106 - 22s - loss: -6.2586e-01 - val_loss: -6.4935e-01 - lr: 6.2500e-04
Epoch 82/100
106/106 - 22s - loss: -6.2570e-01 - val_loss: -6.4474e

Epoch 62/100
106/106 - 22s - loss: -6.2418e-01 - val_loss: -6.8056e-01 - lr: 1.2500e-04
Epoch 63/100
106/106 - 22s - loss: -6.2282e-01 - val_loss: -6.8584e-01 - lr: 1.2500e-04
Epoch 64/100
106/106 - 22s - loss: -6.2639e-01 - val_loss: -6.1851e-01 - lr: 1.2500e-04
Epoch 65/100
106/106 - 22s - loss: -6.2520e-01 - val_loss: -6.3868e-01 - lr: 1.2500e-04
Epoch 66/100
106/106 - 22s - loss: -6.2727e-01 - val_loss: -6.5931e-01 - lr: 1.2500e-04
Epoch 67/100
106/106 - 22s - loss: -6.2669e-01 - val_loss: -6.6967e-01 - lr: 1.2500e-04
Epoch 68/100
106/106 - 22s - loss: -6.2961e-01 - val_loss: -6.5280e-01 - lr: 1.2500e-04
Epoch 69/100
106/106 - 22s - loss: -6.2499e-01 - val_loss: -6.5423e-01 - lr: 1.2500e-04
Epoch 70/100
106/106 - 22s - loss: -6.2924e-01 - val_loss: -6.7414e-01 - lr: 1.2500e-04
Epoch 71/100
106/106 - 22s - loss: -6.3349e-01 - val_loss: -6.5850e-01 - lr: 1.2500e-04
Epoch 72/100
106/106 - 22s - loss: -6.2962e-01 - val_loss: -6.8249e-01 - lr: 1.2500e-04
Epoch 73/100

Epoch 00073: Reduc

Epoch 50/100
106/106 - 22s - loss: -5.9301e-01 - val_loss: -6.1770e-01 - lr: 1.2500e-05
Epoch 51/100
106/106 - 22s - loss: -5.9571e-01 - val_loss: -6.4333e-01 - lr: 1.2500e-05
Epoch 52/100
106/106 - 22s - loss: -5.9139e-01 - val_loss: -6.4831e-01 - lr: 1.2500e-05
Epoch 53/100
106/106 - 22s - loss: -5.9676e-01 - val_loss: -6.3475e-01 - lr: 1.2500e-05
Epoch 54/100
106/106 - 22s - loss: -6.0056e-01 - val_loss: -5.8326e-01 - lr: 1.2500e-05
Epoch 55/100
106/106 - 22s - loss: -5.9645e-01 - val_loss: -6.7171e-01 - lr: 1.2500e-05
Epoch 56/100
106/106 - 22s - loss: -5.9786e-01 - val_loss: -5.9151e-01 - lr: 1.2500e-05
Epoch 57/100
106/106 - 22s - loss: -5.9368e-01 - val_loss: -6.0745e-01 - lr: 1.2500e-05
Epoch 58/100

Epoch 00058: ReduceLROnPlateau reducing learning rate to 6.24999984211172e-06.
106/106 - 22s - loss: -5.9573e-01 - val_loss: -6.5398e-01 - lr: 1.2500e-05
Epoch 59/100
106/106 - 22s - loss: -6.0220e-01 - val_loss: -6.5018e-01 - lr: 6.2500e-06
Epoch 60/100
106/106 - 22s - loss: -5.99

In [ ]:
val_loss_learning = np.load(main_dir+"parameters_analysis/learning_rate.npy")
epochs = range(5,101,5)
mean_val_loss = np.zeros((4,20))
for i in range(20):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_learning[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_learning[1,i:i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_learning[2,i:i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_learning[3,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'learning rate = 0.1')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'learning rate = 0.01')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'learning rate = 0.001')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'learning rate = 0.0001')

plt.title('Evolution of the validation loss with different learning rates ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### UNFREEZING TEST

We test here the amount of layers that we keep frozen during unfreezing phase.

In [15]:
depth = [0,6,10]
val_loss_unfreeze = np.zeros((3,100))

In [16]:


X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=12) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

for layer in long_model.layers[:-9]:
    layer.trainable = False

long_model.compile(optimizer = optimizer_class_name, loss=loss)

K.set_value(long_model.optimizer.learning_rate,0.001)

#fit the long model
history = train_model(model=long_model,
        path2save=config["path2save_transferlearning"],
        model_name='pretraining_unfreeze_test',
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=100,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )

for i in range(3):
    pretrained_model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/transferLearned_models/best_pretraining_unfreeze_test.h5')
    for layer in pretrained_model.layers:
        layer.trainable = True
    for layer in pretrained_model.layers[:depth[i]]:
        layer.trainable = False
    
    pretrained_model.compile(optimizer = optimizer_class_name, loss=loss)
    K.set_value(pretrained_model.optimizer.learning_rate,0.00001)
    print(pretrained_model.summary())
    
    history = train_model(model=pretrained_model,
        path2save=config["path2save_finetuned"],
        model_name=_.join(['unfreeze_test_depth',str(i)]),
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=100,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )
    val_loss_unfreeze[i,:] = history.history['val_loss']
    



    
np.save(main_dir+"parameters_analysis/unfreeze_2.npy",val_loss_unfreeze)

/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-10/t2/pm-10_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-13/t2/pm-13_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-1/t2/pm-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-3/t2/iv-3_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-14/t2/pm-14_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-6/t2/pm-6_t2_norm.nii.gz
/home/nidebroux/data_mask/prepro

Epoch 75/100
100/100 - 21s - loss: -6.3365e-01 - val_loss: -6.1399e-01 - lr: 1.2500e-04
Epoch 76/100
100/100 - 21s - loss: -6.3067e-01 - val_loss: -6.6841e-01 - lr: 1.2500e-04
Epoch 77/100
100/100 - 21s - loss: -6.3228e-01 - val_loss: -6.5645e-01 - lr: 1.2500e-04
Epoch 78/100
100/100 - 21s - loss: -6.3460e-01 - val_loss: -6.0313e-01 - lr: 1.2500e-04
Epoch 79/100
100/100 - 21s - loss: -6.3447e-01 - val_loss: -6.2857e-01 - lr: 1.2500e-04
Epoch 80/100
100/100 - 21s - loss: -6.3577e-01 - val_loss: -6.7679e-01 - lr: 1.2500e-04
Epoch 81/100

Epoch 00081: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
100/100 - 21s - loss: -6.3886e-01 - val_loss: -6.8448e-01 - lr: 1.2500e-04
Epoch 82/100
100/100 - 21s - loss: -6.4157e-01 - val_loss: -6.3709e-01 - lr: 6.2500e-05
Epoch 83/100
100/100 - 21s - loss: -6.4235e-01 - val_loss: -6.5970e-01 - lr: 6.2500e-05
Epoch 84/100
100/100 - 21s - loss: -6.3932e-01 - val_loss: -6.5694e-01 - lr: 6.2500e-05
Epoch 85/100
100/100 - 21s - loss: -6.40

Epoch 5/100
100/100 - 37s - loss: -6.0362e-01 - val_loss: -5.2673e-01 - lr: 1.0000e-05
Epoch 6/100
100/100 - 37s - loss: -6.1339e-01 - val_loss: -6.0864e-01 - lr: 1.0000e-05
Epoch 7/100
100/100 - 37s - loss: -6.1356e-01 - val_loss: -6.1135e-01 - lr: 1.0000e-05
Epoch 8/100
100/100 - 37s - loss: -6.2229e-01 - val_loss: -6.3226e-01 - lr: 1.0000e-05
Epoch 9/100
100/100 - 37s - loss: -6.2728e-01 - val_loss: -6.4263e-01 - lr: 1.0000e-05
Epoch 10/100
100/100 - 37s - loss: -6.2913e-01 - val_loss: -6.4372e-01 - lr: 1.0000e-05
Epoch 11/100
100/100 - 37s - loss: -6.3428e-01 - val_loss: -6.3728e-01 - lr: 1.0000e-05
Epoch 12/100
100/100 - 37s - loss: -6.3377e-01 - val_loss: -6.4996e-01 - lr: 1.0000e-05
Epoch 13/100
100/100 - 37s - loss: -6.3493e-01 - val_loss: -6.6189e-01 - lr: 1.0000e-05
Epoch 14/100
100/100 - 37s - loss: -6.4181e-01 - val_loss: -6.5659e-01 - lr: 1.0000e-05
Epoch 15/100
100/100 - 37s - loss: -6.4505e-01 - val_loss: -6.1942e-01 - lr: 1.0000e-05
Epoch 16/100
100/100 - 37s - loss: -6

Epoch 93/100
100/100 - 37s - loss: -6.7838e-01 - val_loss: -6.2970e-01 - lr: 7.8125e-08
Epoch 94/100
100/100 - 37s - loss: -6.7467e-01 - val_loss: -6.8483e-01 - lr: 7.8125e-08
Epoch 95/100
100/100 - 37s - loss: -6.7770e-01 - val_loss: -6.8437e-01 - lr: 7.8125e-08
Epoch 96/100
100/100 - 37s - loss: -6.8039e-01 - val_loss: -6.6969e-01 - lr: 7.8125e-08
Epoch 97/100
100/100 - 37s - loss: -6.7735e-01 - val_loss: -6.5413e-01 - lr: 7.8125e-08
Epoch 98/100
100/100 - 37s - loss: -6.8394e-01 - val_loss: -6.9275e-01 - lr: 7.8125e-08
Epoch 99/100
100/100 - 37s - loss: -6.7880e-01 - val_loss: -6.5984e-01 - lr: 7.8125e-08
Epoch 100/100
100/100 - 37s - loss: -6.7506e-01 - val_loss: -6.8730e-01 - lr: 7.8125e-08
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Model: "long_surgery_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
in

Epoch 24/100
100/100 - 31s - loss: -6.4064e-01 - val_loss: -6.8734e-01 - lr: 5.0000e-06
Epoch 25/100
100/100 - 31s - loss: -6.3957e-01 - val_loss: -6.4235e-01 - lr: 5.0000e-06
Epoch 26/100
100/100 - 31s - loss: -6.4018e-01 - val_loss: -6.3870e-01 - lr: 5.0000e-06
Epoch 27/100
100/100 - 31s - loss: -6.4411e-01 - val_loss: -6.7405e-01 - lr: 5.0000e-06
Epoch 28/100
100/100 - 31s - loss: -6.4163e-01 - val_loss: -6.4865e-01 - lr: 5.0000e-06
Epoch 29/100
100/100 - 31s - loss: -6.4085e-01 - val_loss: -6.9246e-01 - lr: 5.0000e-06
Epoch 30/100
100/100 - 31s - loss: -6.4293e-01 - val_loss: -6.5201e-01 - lr: 5.0000e-06
Epoch 31/100
100/100 - 31s - loss: -6.4473e-01 - val_loss: -6.8040e-01 - lr: 5.0000e-06
Epoch 32/100
100/100 - 31s - loss: -6.3937e-01 - val_loss: -6.3404e-01 - lr: 5.0000e-06
Epoch 33/100
100/100 - 31s - loss: -6.4849e-01 - val_loss: -6.7481e-01 - lr: 5.0000e-06
Epoch 34/100
100/100 - 31s - loss: -6.4912e-01 - val_loss: -6.4523e-01 - lr: 5.0000e-06
Epoch 35/100
100/100 - 31s - los

100/100 - 30s - loss: -4.3118e-01 - val_loss: -4.8639e-01 - lr: 1.0000e-05
Epoch 2/100
100/100 - 29s - loss: -5.1273e-01 - val_loss: -5.6165e-01 - lr: 1.0000e-05
Epoch 3/100
100/100 - 29s - loss: -5.2334e-01 - val_loss: -5.8524e-01 - lr: 1.0000e-05
Epoch 4/100
100/100 - 29s - loss: -5.5351e-01 - val_loss: -5.6223e-01 - lr: 1.0000e-05
Epoch 5/100
100/100 - 30s - loss: -5.6527e-01 - val_loss: -5.8996e-01 - lr: 1.0000e-05
Epoch 6/100
100/100 - 30s - loss: -5.6924e-01 - val_loss: -6.0642e-01 - lr: 1.0000e-05
Epoch 7/100
100/100 - 29s - loss: -5.8292e-01 - val_loss: -5.6345e-01 - lr: 1.0000e-05
Epoch 8/100
100/100 - 30s - loss: -5.8879e-01 - val_loss: -6.1073e-01 - lr: 1.0000e-05
Epoch 9/100
100/100 - 29s - loss: -5.9337e-01 - val_loss: -5.9072e-01 - lr: 1.0000e-05
Epoch 10/100
100/100 - 29s - loss: -5.9829e-01 - val_loss: -6.0853e-01 - lr: 1.0000e-05
Epoch 11/100
100/100 - 30s - loss: -6.0305e-01 - val_loss: -6.2047e-01 - lr: 1.0000e-05
Epoch 12/100
100/100 - 29s - loss: -6.0170e-01 - val_

Epoch 91/100
100/100 - 29s - loss: -6.6125e-01 - val_loss: -6.3184e-01 - lr: 6.2500e-07
Epoch 92/100
100/100 - 29s - loss: -6.5996e-01 - val_loss: -6.8398e-01 - lr: 6.2500e-07
Epoch 93/100
100/100 - 29s - loss: -6.5442e-01 - val_loss: -6.3996e-01 - lr: 6.2500e-07
Epoch 94/100
100/100 - 29s - loss: -6.5596e-01 - val_loss: -6.4215e-01 - lr: 6.2500e-07
Epoch 95/100
100/100 - 30s - loss: -6.5792e-01 - val_loss: -6.5991e-01 - lr: 6.2500e-07
Epoch 96/100
100/100 - 29s - loss: -6.6279e-01 - val_loss: -6.5148e-01 - lr: 6.2500e-07
Epoch 97/100

Epoch 00097: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.
100/100 - 30s - loss: -6.5702e-01 - val_loss: -6.6658e-01 - lr: 6.2500e-07
Epoch 98/100
100/100 - 29s - loss: -6.5894e-01 - val_loss: -6.4948e-01 - lr: 3.1250e-07
Epoch 99/100
100/100 - 29s - loss: -6.5971e-01 - val_loss: -6.4967e-01 - lr: 3.1250e-07
Epoch 100/100
100/100 - 29s - loss: -6.5205e-01 - val_loss: -6.1634e-01 - lr: 3.1250e-07


In [ ]:
val_loss_unfreeze = np.load(main_dir+"parameters_analysis/unfreeze_2.npy")
epochs = range(5,101,5)
mean_val_loss = np.zeros((3,20))
for i in range(20):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_unfreeze[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_unfreeze[1,i:i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_unfreeze[2,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'frozen parameters = 0')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'frozen parameters = 32 112')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'frozen parameters = 94 560')

plt.title('Evolution of the validation loss for different unfreezing possibilities')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### UNFREEZING --> LEARNING RATE TEST

In [11]:
learning_array = [0.0001,0.00001,0.000001]
val_loss_learning = np.zeros((4,100))

In [13]:
X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=12) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

for layer in long_model.layers[:-9]:
    layer.trainable = False

long_model.compile(optimizer = optimizer_class_name, loss=loss)

K.set_value(long_model.optimizer.learning_rate,0.001)

#fit the long model
history = train_model(model=long_model,
        path2save=config["path2save_transferlearning"],
        model_name='pretraining_unfreeze_test',
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=100,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )
val_loss_learning[0,:] = history.history['val_loss']





for i in range(3):
    pretrained_model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/transferLearned_models/best_pretraining_unfreeze_test.h5')
    for layer in pretrained_model.layers:
        layer.trainable = True
    
    pretrained_model.compile(optimizer = optimizer_class_name, loss=loss)
    K.set_value(pretrained_model.optimizer.learning_rate,learning_array[i])
    
    history = train_model(model=pretrained_model,
        path2save=config["path2save_finetuned"],
        model_name=_.join(['unfreeze_test_learning',str(i)]),
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=100,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )
    val_loss_learning[i+1,:] = history.history['val_loss']
    



    
np.save(main_dir+"parameters_analysis/unfreeze_learning_rate.npy",val_loss_learning)

/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-1/t2/pm-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-6/t2/iv-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-3/t2/iv-3_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-14/t2/pm-14_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-6/t2/pm-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-5/t2/pm-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess

105/105 - 22s - loss: -6.2735e-01 - val_loss: -6.4227e-01 - lr: 6.2500e-05
Epoch 75/100
105/105 - 22s - loss: -6.2971e-01 - val_loss: -6.4476e-01 - lr: 6.2500e-05
Epoch 76/100
105/105 - 22s - loss: -6.3160e-01 - val_loss: -6.3764e-01 - lr: 6.2500e-05
Epoch 77/100
105/105 - 22s - loss: -6.2426e-01 - val_loss: -6.3978e-01 - lr: 6.2500e-05
Epoch 78/100
105/105 - 22s - loss: -6.2779e-01 - val_loss: -6.3376e-01 - lr: 6.2500e-05
Epoch 79/100
105/105 - 22s - loss: -6.3360e-01 - val_loss: -6.3588e-01 - lr: 6.2500e-05
Epoch 80/100
105/105 - 22s - loss: -6.2938e-01 - val_loss: -6.3565e-01 - lr: 6.2500e-05
Epoch 81/100

Epoch 00081: ReduceLROnPlateau reducing learning rate to 3.125000148429535e-05.
105/105 - 22s - loss: -6.2797e-01 - val_loss: -6.5004e-01 - lr: 6.2500e-05
Epoch 82/100
105/105 - 22s - loss: -6.3058e-01 - val_loss: -6.2713e-01 - lr: 3.1250e-05
Epoch 83/100
105/105 - 22s - loss: -6.3389e-01 - val_loss: -6.4322e-01 - lr: 3.1250e-05
Epoch 84/100
105/105 - 22s - loss: -6.3337e-01 - val

Epoch 63/100
105/105 - 38s - loss: -7.6371e-01 - val_loss: -6.2169e-01 - lr: 2.5000e-05
Epoch 64/100
105/105 - 38s - loss: -7.6801e-01 - val_loss: -6.6045e-01 - lr: 2.5000e-05
Epoch 65/100
105/105 - 38s - loss: -7.6352e-01 - val_loss: -6.8719e-01 - lr: 2.5000e-05
Epoch 66/100
105/105 - 38s - loss: -7.6564e-01 - val_loss: -7.4761e-01 - lr: 2.5000e-05
Epoch 67/100

Epoch 00067: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-05.
105/105 - 38s - loss: -7.6777e-01 - val_loss: -6.5335e-01 - lr: 2.5000e-05
Epoch 68/100
105/105 - 38s - loss: -7.6519e-01 - val_loss: -7.1406e-01 - lr: 1.2500e-05
Epoch 69/100
105/105 - 38s - loss: -7.6704e-01 - val_loss: -7.0483e-01 - lr: 1.2500e-05
Epoch 70/100
105/105 - 38s - loss: -7.6931e-01 - val_loss: -6.6099e-01 - lr: 1.2500e-05
Epoch 71/100
105/105 - 38s - loss: -7.6774e-01 - val_loss: -6.3153e-01 - lr: 1.2500e-05
Epoch 72/100
105/105 - 38s - loss: -7.6851e-01 - val_loss: -7.0397e-01 - lr: 1.2500e-05
Epoch 73/100
105/105 - 38s - loss: -7.6

Epoch 50/100
105/105 - 38s - loss: -6.7006e-01 - val_loss: -7.0572e-01 - lr: 2.5000e-06
Epoch 51/100
105/105 - 38s - loss: -6.7199e-01 - val_loss: -6.7620e-01 - lr: 2.5000e-06
Epoch 52/100
105/105 - 38s - loss: -6.6936e-01 - val_loss: -6.7058e-01 - lr: 2.5000e-06
Epoch 53/100
105/105 - 38s - loss: -6.7071e-01 - val_loss: -6.9084e-01 - lr: 2.5000e-06
Epoch 54/100

Epoch 00054: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.
105/105 - 38s - loss: -6.7187e-01 - val_loss: -7.2083e-01 - lr: 2.5000e-06
Epoch 55/100
105/105 - 38s - loss: -6.7204e-01 - val_loss: -6.8301e-01 - lr: 1.2500e-06
Epoch 56/100
105/105 - 38s - loss: -6.7034e-01 - val_loss: -7.0281e-01 - lr: 1.2500e-06
Epoch 57/100
105/105 - 38s - loss: -6.7510e-01 - val_loss: -6.2794e-01 - lr: 1.2500e-06
Epoch 58/100
105/105 - 38s - loss: -6.6985e-01 - val_loss: -6.6843e-01 - lr: 1.2500e-06
Epoch 59/100
105/105 - 38s - loss: -6.7511e-01 - val_loss: -7.5054e-01 - lr: 1.2500e-06
Epoch 60/100
105/105 - 38s - loss: -6.7

Epoch 38/100
105/105 - 38s - loss: -6.0351e-01 - val_loss: -6.0669e-01 - lr: 1.0000e-06
Epoch 39/100
105/105 - 38s - loss: -6.0933e-01 - val_loss: -6.2129e-01 - lr: 1.0000e-06
Epoch 40/100
105/105 - 38s - loss: -6.0303e-01 - val_loss: -6.0300e-01 - lr: 1.0000e-06
Epoch 41/100
105/105 - 38s - loss: -6.0728e-01 - val_loss: -6.2804e-01 - lr: 1.0000e-06
Epoch 42/100
105/105 - 38s - loss: -6.0284e-01 - val_loss: -6.3057e-01 - lr: 1.0000e-06
Epoch 43/100
105/105 - 38s - loss: -6.1191e-01 - val_loss: -6.0893e-01 - lr: 1.0000e-06
Epoch 44/100

Epoch 00044: ReduceLROnPlateau reducing learning rate to 4.999999987376214e-07.
105/105 - 38s - loss: -6.0915e-01 - val_loss: -6.3801e-01 - lr: 1.0000e-06
Epoch 45/100
105/105 - 38s - loss: -6.1072e-01 - val_loss: -6.5190e-01 - lr: 5.0000e-07
Epoch 46/100
105/105 - 38s - loss: -6.0866e-01 - val_loss: -6.4105e-01 - lr: 5.0000e-07
Epoch 47/100
105/105 - 38s - loss: -6.0872e-01 - val_loss: -6.4015e-01 - lr: 5.0000e-07
Epoch 48/100
105/105 - 38s - loss: -6.1

NameError: name 'val_loss_unfreeze' is not defined

In [ ]:
val_loss_learning = np.load(main_dir+"parameters_analysis/unfreeze_learning_rate.npy")
epochs = range(5,101,5)
mean_val_loss = np.zeros((4,20))
for i in range(20):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_learning[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_learning[1,i:i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_learning[2,i:i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_learning[3,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'First training results (before unfreezing)')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'learning rate = 0.0001')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'learning rate = 0.00001')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'learning rate = 0.000001')

plt.title('Evolution of the validation loss with different learning rates during unfreezing phase ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### DIRECT RETRAINING


We test here which layers we freeze in case of direct retraining.


In [11]:
depth = [0,6,10,16]
val_loss_unfreeze = np.zeros((4,100))

In [14]:
X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=12) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)


for i in range(4):
    model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')
    for layer in model.layers:
        layer.trainable = True
    for layer in model.layers[:depth[i]]:
        layer.trainable = False
    
    model.compile(optimizer = optimizer_class_name, loss=loss)
    K.set_value(model.optimizer.learning_rate,0.00001)
    print(model.summary())
    
    history = train_model(model=model,
        path2save=config["path2save_retrained"],
        model_name=_.join(['retrained',str(i)]),
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=100,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )
    val_loss_unfreeze[i,:] = history.history['val_loss']
    



    
np.save(main_dir+"parameters_analysis/retraining.npy",val_loss_unfreeze)

/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-10/t2/pm-10_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-1/t2/pm-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-6/t2/iv-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-3/t2/iv-3_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-14/t2/pm-14_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-9/t2/iv-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-6/t2/pm-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-5/t2/pm-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preproce

Epoch 16/100
107/107 - 39s - loss: -6.7073e-01 - val_loss: -6.0992e-01 - lr: 1.0000e-05
Epoch 17/100
107/107 - 39s - loss: -6.7493e-01 - val_loss: -5.3498e-01 - lr: 1.0000e-05
Epoch 18/100
107/107 - 39s - loss: -6.7570e-01 - val_loss: -6.2035e-01 - lr: 1.0000e-05
Epoch 19/100
107/107 - 39s - loss: -6.7858e-01 - val_loss: -6.0118e-01 - lr: 1.0000e-05
Epoch 20/100
107/107 - 39s - loss: -6.7883e-01 - val_loss: -6.0508e-01 - lr: 1.0000e-05
Epoch 21/100
107/107 - 39s - loss: -6.7733e-01 - val_loss: -5.5824e-01 - lr: 1.0000e-05
Epoch 22/100
107/107 - 39s - loss: -6.8316e-01 - val_loss: -5.9373e-01 - lr: 1.0000e-05
Epoch 23/100
107/107 - 39s - loss: -6.8362e-01 - val_loss: -5.6031e-01 - lr: 1.0000e-05
Epoch 24/100
107/107 - 39s - loss: -6.8991e-01 - val_loss: -6.2290e-01 - lr: 1.0000e-05
Epoch 25/100
107/107 - 39s - loss: -6.8604e-01 - val_loss: -5.8988e-01 - lr: 1.0000e-05
Epoch 26/100
107/107 - 39s - loss: -6.9279e-01 - val_loss: -5.9743e-01 - lr: 1.0000e-05
Epoch 27/100
107/107 - 39s - los

107/107 - 33s - loss: -3.4518e-01 - val_loss: -2.1472e-01 - lr: 1.0000e-05
Epoch 2/100
107/107 - 33s - loss: -4.8154e-01 - val_loss: -5.3663e-01 - lr: 1.0000e-05
Epoch 3/100
107/107 - 33s - loss: -5.2825e-01 - val_loss: -5.3803e-01 - lr: 1.0000e-05
Epoch 4/100
107/107 - 33s - loss: -5.4927e-01 - val_loss: -5.8076e-01 - lr: 1.0000e-05
Epoch 5/100
107/107 - 33s - loss: -5.6553e-01 - val_loss: -6.1155e-01 - lr: 1.0000e-05
Epoch 6/100
107/107 - 33s - loss: -5.7700e-01 - val_loss: -5.3510e-01 - lr: 1.0000e-05
Epoch 7/100
107/107 - 33s - loss: -5.8478e-01 - val_loss: -5.5471e-01 - lr: 1.0000e-05
Epoch 8/100
107/107 - 33s - loss: -5.9323e-01 - val_loss: -5.6631e-01 - lr: 1.0000e-05
Epoch 9/100
107/107 - 33s - loss: -5.9508e-01 - val_loss: -5.7407e-01 - lr: 1.0000e-05
Epoch 10/100
107/107 - 33s - loss: -6.0432e-01 - val_loss: -6.0452e-01 - lr: 1.0000e-05
Epoch 11/100
107/107 - 33s - loss: -6.1009e-01 - val_loss: -5.9376e-01 - lr: 1.0000e-05
Epoch 12/100
107/107 - 33s - loss: -6.0973e-01 - val_

Epoch 90/100
107/107 - 33s - loss: -6.7194e-01 - val_loss: -5.7023e-01 - lr: 3.1250e-07
Epoch 91/100
107/107 - 33s - loss: -6.7625e-01 - val_loss: -5.4479e-01 - lr: 3.1250e-07
Epoch 92/100
107/107 - 33s - loss: -6.7227e-01 - val_loss: -6.2350e-01 - lr: 3.1250e-07
Epoch 93/100
107/107 - 33s - loss: -6.7283e-01 - val_loss: -5.6033e-01 - lr: 3.1250e-07
Epoch 94/100
107/107 - 33s - loss: -6.6970e-01 - val_loss: -5.9947e-01 - lr: 3.1250e-07
Epoch 95/100
107/107 - 33s - loss: -6.7211e-01 - val_loss: -6.0914e-01 - lr: 3.1250e-07
Epoch 96/100

Epoch 00096: ReduceLROnPlateau reducing learning rate to 1.56249996052793e-07.
107/107 - 33s - loss: -6.7272e-01 - val_loss: -5.7263e-01 - lr: 3.1250e-07
Epoch 97/100
107/107 - 33s - loss: -6.7300e-01 - val_loss: -6.1579e-01 - lr: 1.5625e-07
Epoch 98/100
107/107 - 33s - loss: -6.7152e-01 - val_loss: -5.8943e-01 - lr: 1.5625e-07
Epoch 99/100
107/107 - 33s - loss: -6.7048e-01 - val_loss: -6.0670e-01 - lr: 1.5625e-07
Epoch 100/100
107/107 - 33s - loss: -6.7

Epoch 21/100
107/107 - 32s - loss: -5.8560e-01 - val_loss: -6.1731e-01 - lr: 1.0000e-05
Epoch 22/100
107/107 - 31s - loss: -5.9021e-01 - val_loss: -5.5051e-01 - lr: 1.0000e-05
Epoch 23/100
107/107 - 31s - loss: -5.9596e-01 - val_loss: -5.3282e-01 - lr: 1.0000e-05
Epoch 24/100
107/107 - 32s - loss: -5.9438e-01 - val_loss: -6.3580e-01 - lr: 1.0000e-05
Epoch 25/100
107/107 - 31s - loss: -5.9369e-01 - val_loss: -5.0498e-01 - lr: 1.0000e-05
Epoch 26/100
107/107 - 31s - loss: -5.9522e-01 - val_loss: -5.6120e-01 - lr: 1.0000e-05
Epoch 27/100
107/107 - 31s - loss: -5.9945e-01 - val_loss: -5.4602e-01 - lr: 1.0000e-05
Epoch 28/100
107/107 - 31s - loss: -6.0555e-01 - val_loss: -5.6030e-01 - lr: 1.0000e-05
Epoch 29/100
107/107 - 31s - loss: -6.1093e-01 - val_loss: -5.6099e-01 - lr: 1.0000e-05
Epoch 30/100
107/107 - 31s - loss: -6.0599e-01 - val_loss: -5.7932e-01 - lr: 1.0000e-05
Epoch 31/100
107/107 - 31s - loss: -6.0311e-01 - val_loss: -5.9124e-01 - lr: 1.0000e-05
Epoch 32/100
107/107 - 31s - los

107/107 - 24s - loss: -1.1662e-01 - val_loss: -5.9976e-02 - lr: 1.0000e-05
Epoch 2/100
107/107 - 22s - loss: -2.6876e-01 - val_loss: -3.1378e-01 - lr: 1.0000e-05
Epoch 3/100
107/107 - 22s - loss: -3.4546e-01 - val_loss: -4.0928e-01 - lr: 1.0000e-05
Epoch 4/100
107/107 - 22s - loss: -3.7874e-01 - val_loss: -4.1595e-01 - lr: 1.0000e-05
Epoch 5/100
107/107 - 22s - loss: -3.9510e-01 - val_loss: -4.0263e-01 - lr: 1.0000e-05
Epoch 6/100
107/107 - 22s - loss: -4.1780e-01 - val_loss: -4.0998e-01 - lr: 1.0000e-05
Epoch 7/100
107/107 - 22s - loss: -4.2172e-01 - val_loss: -4.3792e-01 - lr: 1.0000e-05
Epoch 8/100
107/107 - 22s - loss: -4.4267e-01 - val_loss: -4.3058e-01 - lr: 1.0000e-05
Epoch 9/100
107/107 - 23s - loss: -4.4802e-01 - val_loss: -4.6470e-01 - lr: 1.0000e-05
Epoch 10/100
107/107 - 22s - loss: -4.5101e-01 - val_loss: -4.3277e-01 - lr: 1.0000e-05
Epoch 11/100
107/107 - 22s - loss: -4.6254e-01 - val_loss: -4.5591e-01 - lr: 1.0000e-05
Epoch 12/100
107/107 - 23s - loss: -4.6812e-01 - val_

Epoch 90/100
107/107 - 22s - loss: -5.6941e-01 - val_loss: -5.1878e-01 - lr: 6.2500e-07
Epoch 91/100
107/107 - 22s - loss: -5.6502e-01 - val_loss: -5.5349e-01 - lr: 6.2500e-07
Epoch 92/100
107/107 - 22s - loss: -5.7004e-01 - val_loss: -5.4974e-01 - lr: 6.2500e-07
Epoch 93/100
107/107 - 22s - loss: -5.6384e-01 - val_loss: -5.5635e-01 - lr: 6.2500e-07
Epoch 94/100
107/107 - 22s - loss: -5.7107e-01 - val_loss: -5.3253e-01 - lr: 6.2500e-07
Epoch 95/100

Epoch 00095: ReduceLROnPlateau reducing learning rate to 3.12499992105586e-07.
107/107 - 22s - loss: -5.7167e-01 - val_loss: -5.3037e-01 - lr: 6.2500e-07
Epoch 96/100
107/107 - 22s - loss: -5.6823e-01 - val_loss: -5.6540e-01 - lr: 3.1250e-07
Epoch 97/100
107/107 - 22s - loss: -5.6766e-01 - val_loss: -5.4148e-01 - lr: 3.1250e-07
Epoch 98/100
107/107 - 22s - loss: -5.6981e-01 - val_loss: -5.3221e-01 - lr: 3.1250e-07
Epoch 99/100
107/107 - 22s - loss: -5.6970e-01 - val_loss: -5.4907e-01 - lr: 3.1250e-07
Epoch 100/100
107/107 - 22s - loss: -5.7

In [ ]:
val_loss_unfreeze = np.load(main_dir+"parameters_analysis/retraining.npy")
epochs = range(5,101,5)
mean_val_loss = np.zeros((4,20))
for i in range(20):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_unfreeze[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_unfreeze[1,i:i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_unfreeze[2,i:i+5]))
    mean_val_loss[3,i]=np.abs(np.mean(val_loss_unfreeze[3,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'frozen parameters = 0')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'frozen parameters = 32 112')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'frozen parameters = 94 560')
plt.plot(epochs,mean_val_loss[3,:],'o--',label = 'frozen parameters = 219 024')

plt.title('Evolution of the validation loss for different unfreezing possibilities starting from the original model')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### DATA AUGMENTATION TEST

In [7]:
augment_array = [False,True]
flip_array = [False,True]
val_loss_augment = np.zeros((2,70))
train_loss_augment = np.zeros((2,70))

In [8]:


for i in range(2):
    
    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=16) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)

    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=5,
                                                    augment=augment_array[i],
                                                    augment_flip=flip_array[i])


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=2,
                                                    augment=False,
                                                    augment_flip=False)

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
    long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
    long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
    long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
    long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
    long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
    long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
    long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

    long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

    for layer in long_model.layers[:-9]:
        layer.trainable = False

    long_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(long_model.optimizer.learning_rate,0.001)

    #fit the long model
    history = train_model(model=long_model,
            path2save=config["path2save_retraining"],
            model_name='_'.join(["augment_test",str(i)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=70,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_augment[i,:] = history.history['val_loss']
    train_loss_augment[i,:] = history.history['loss']
    
np.save(main_dir+"parameters_analysis/augment_val.npy",val_loss_augment)
np.save(main_dir+"parameters_analysis/augment_train.npy",train_loss_augment)

/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-10/t2/pm-10_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-13/t2/pm-13_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-1/t2/pm-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-6/t2/iv-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-3/t2/iv-3_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-14/t2/pm-14_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-9/t2/iv-9_t2_norm.nii.gz
/home/nidebroux/data_mask/prepro

/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-9/t2/iv-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-12/t2/pm-12_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-7/t2/iv-7_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-2/t2/pm-2_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-6/t2/pm-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-5/t2/pm-5_t2_norm.nii.gz
Epoch 1/70
81/81 - 17s - loss: -2.9280e-01 - val_loss: -2.6007e-01 - lr: 0.0010
Epoch 2/70
81/81 - 17s - loss: -3.8883e-01 - val_loss: -3.9329e-01 - lr: 0.0010
Epoch 3/70
81/81 - 17s - loss: -4.1730e-01 - val_loss: -3.1195e-01 - lr: 0.0010
Epoch 4/70
81/81 - 17s - loss: -4.7139

In [ ]:
val_loss_augment = np.load(main_dir+"parameters_analysis/augment_val.npy")
loss_augment = np.load(main_dir+"parameters_analysis/augment_train.npy")
epochs = range(5,71,5)
mean_val_loss = np.zeros((2,14))
mean_loss = np.zeros((2,14))
for i in range(14):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_augment[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_augment[1,i:i+5]))
    mean_loss[0,i]=np.abs(np.mean(loss_augment[0,i:i+5]))
    mean_loss[1,i]=np.abs(np.mean(loss_augment[1,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'validation loss without data augmentation')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'validation loss with data augmentation')
plt.plot(epochs,mean_loss[0,:],'o--',label = 'train loss without data augmentation')
plt.plot(epochs,mean_loss[1,:],'o--',label = 'train loss with data augmentation')

plt.title('Evolution of the validation and train loss with and without data augmentation')
# show a legend on the plot
plt.ylabel('Mean of the validation/train loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### SECOND OVERLAP TEST

In [7]:
overlap_array = [12,6,3]
val_loss_overlap = np.zeros((3,50))
loss_overlap = np.zeros((3,50))

In [8]:
for i in range(3):

    X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=overlap_array[i]) 
    X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
    X_train, y_train = shuffle(X_train, y_train, random_state=2611)
    
    X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                        patch_shape=config["patch_size"],
                                        overlap=0)
    X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)
    
    

    train_generator, nb_train_steps = get_training_and_validation_generators(
                                                    [X_train, y_train],
                                                    batch_size=5,
                                                    augment=True,
                                                    augment_flip=True)


    validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                    [X_valid, y_valid],
                                                    batch_size=2,
                                                    augment=False,
                                                    augment_flip=False)

    init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
    long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
    long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
    long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
    long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
    long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
    long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
    long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
    long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

    long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

    for layer in long_model.layers[:-9]:
        layer.trainable = False

    long_model.compile(optimizer = optimizer_class_name, loss=loss)

    K.set_value(long_model.optimizer.learning_rate,0.001)

    #fit the long model
    history = train_model(model=long_model,
            path2save=config["path2save_transferlearning"],
            model_name='_'.join(["second_overlap_test",str(i)]),
            training_generator=train_generator,
            validation_generator=validation_generator,
            steps_per_epoch=nb_train_steps,
            validation_steps=nb_valid_steps,
            n_epochs=50,
            learning_rate_drop=0.5,
            learning_rate_patience=10
           )

    val_loss_overlap[i,:] = history.history['val_loss']
    loss_overlap[i,:] = history.history['loss']
    
np.save(main_dir+"parameters_analysis/second_overlap.npy",val_loss_overlap)
np.save(main_dir+"parameters_analysis/second_overlap_train.npy",loss_overlap)

/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-10/t2/pm-10_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-1/t2/pm-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-6/t2/iv-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-9/t2/iv-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-6/t2/pm-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-5/t2/pm-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess

Epoch 11/50
209/209 - 43s - loss: -5.9662e-01 - val_loss: -5.0431e-01 - lr: 0.0010
Epoch 12/50
209/209 - 44s - loss: -5.9412e-01 - val_loss: -4.7272e-01 - lr: 0.0010
Epoch 13/50
209/209 - 45s - loss: -5.9697e-01 - val_loss: -5.1491e-01 - lr: 0.0010
Epoch 14/50
209/209 - 43s - loss: -6.0490e-01 - val_loss: -4.9706e-01 - lr: 0.0010
Epoch 15/50
209/209 - 43s - loss: -6.0515e-01 - val_loss: -5.0011e-01 - lr: 0.0010
Epoch 16/50
209/209 - 44s - loss: -6.1289e-01 - val_loss: -4.5425e-01 - lr: 0.0010
Epoch 17/50
209/209 - 44s - loss: -6.1769e-01 - val_loss: -4.8819e-01 - lr: 0.0010
Epoch 18/50

Epoch 00018: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
209/209 - 43s - loss: -6.2174e-01 - val_loss: -5.0545e-01 - lr: 0.0010
Epoch 19/50
209/209 - 44s - loss: -6.3313e-01 - val_loss: -4.2488e-01 - lr: 5.0000e-04
Epoch 20/50
209/209 - 45s - loss: -6.3767e-01 - val_loss: -5.2431e-01 - lr: 5.0000e-04
Epoch 21/50
209/209 - 43s - loss: -6.3704e-01 - val_loss: -4.9921e-01 - lr: 5.000

Epoch 37/50
415/415 - 86s - loss: -6.8796e-01 - val_loss: -5.5731e-01 - lr: 5.0000e-04
Epoch 38/50
415/415 - 87s - loss: -6.8987e-01 - val_loss: -5.4985e-01 - lr: 5.0000e-04
Epoch 39/50
415/415 - 86s - loss: -6.9166e-01 - val_loss: -5.2421e-01 - lr: 5.0000e-04
Epoch 40/50
415/415 - 86s - loss: -6.9007e-01 - val_loss: -5.4930e-01 - lr: 5.0000e-04
Epoch 41/50
415/415 - 87s - loss: -6.9125e-01 - val_loss: -5.7263e-01 - lr: 5.0000e-04
Epoch 42/50
415/415 - 86s - loss: -6.9269e-01 - val_loss: -5.2018e-01 - lr: 5.0000e-04
Epoch 43/50
415/415 - 87s - loss: -6.9173e-01 - val_loss: -5.1996e-01 - lr: 5.0000e-04
Epoch 44/50
415/415 - 86s - loss: -6.9420e-01 - val_loss: -6.0017e-01 - lr: 5.0000e-04
Epoch 45/50
415/415 - 87s - loss: -6.9488e-01 - val_loss: -4.6779e-01 - lr: 5.0000e-04
Epoch 46/50
415/415 - 86s - loss: -6.9762e-01 - val_loss: -5.4163e-01 - lr: 5.0000e-04
Epoch 47/50
415/415 - 86s - loss: -6.9303e-01 - val_loss: -5.6707e-01 - lr: 5.0000e-04
Epoch 48/50
415/415 - 87s - loss: -6.9398e-

In [ ]:
val_loss_overlap = np.load(main_dir+"parameters_analysis/second_overlap.npy")
epochs = range(5,51,5)
mean_val_loss = np.zeros((3,10))
for i in range(10):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss_overlap[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss_overlap[1,i:i+5]))
    mean_val_loss[2,i]=np.abs(np.mean(val_loss_overlap[2,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'overlap of 12')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'overlap of 6')
plt.plot(epochs,mean_val_loss[2,:],'o--',label = 'overlap of 3')

plt.title('Evolution of the validation loss with different overlap sizes ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()

### TRANSFER LEARNING + UNFREEZING VS DIRECT RETRAINING

In [8]:
val_loss = np.zeros((2,150))

In [9]:
X_train, y_train = load_3Dpatches(fname_lst=training_files,patch_shape=config["patch_size"],overlap=6) 
X_train = normalize_data(X_train, MEAN_TRAIN_T2, STD_TRAIN_T2)
X_train, y_train = shuffle(X_train, y_train, random_state=2611)

X_valid, y_valid = load_3Dpatches(fname_lst=validation_files,
                                    patch_shape=config["patch_size"],
                                    overlap=0)
X_valid = normalize_data(X_valid, MEAN_TRAIN_T2, STD_TRAIN_T2)



train_generator, nb_train_steps = get_training_and_validation_generators(
                                                [X_train, y_train],
                                                batch_size=5,
                                                augment=True,
                                                augment_flip=True)


validation_generator, nb_valid_steps = get_training_and_validation_generators(
                                                [X_valid, y_valid],
                                                batch_size=2,
                                                augment=False,
                                                augment_flip=False)

init = tf.keras.initializers.VarianceScaling(scale = 1.0, mode = 'fan_avg', distribution = 'uniform', seed= None)
long_1 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_1',padding = 'same',kernel_initializer = init)(model.layers[-9].output)
long_2 = keras.layers.BatchNormalization(name = 'long_batch_1')(long_1)
long_3 = keras.layers.Activation('relu',name = 'long_activation_1')(long_2)
long_4 = keras.layers.Conv3D(48,kernel_size=(3,3,3),activation='linear',data_format = 'channels_first', name = 'long_conv3D_2',padding = 'same',kernel_initializer = init)(long_3)
long_5 = keras.layers.BatchNormalization(name = 'long_batch_2')(long_4)
long_6 = keras.layers.Activation('relu',name = 'long_activation_2')(long_5)
long_7 = keras.layers.Conv3D(1,kernel_size=(1,1,1),activation='linear',data_format = 'channels_first', name = 'long_conv3D_3',kernel_initializer = init)(long_6)
long_output = keras.layers.Activation('sigmoid',name = 'long_activation_3')(long_7)

long_model = keras.models.Model(model.input,long_output,name = "long_surgery_model")

for layer in long_model.layers[:-9]:
    layer.trainable = False

long_model.compile(optimizer = optimizer_class_name, loss=loss)

K.set_value(long_model.optimizer.learning_rate,0.001)

#fit the long model
history = train_model(model=long_model,
        path2save=config["path2save_transferlearning"],
        model_name='final_test',
        training_generator=train_generator,
        validation_generator=validation_generator,
        steps_per_epoch=nb_train_steps,
        validation_steps=nb_valid_steps,
        n_epochs=75,
        learning_rate_drop=0.5,
        learning_rate_patience=10
       )

val_loss[0,:75] = history.history['val_loss']

for layer in long_model.layers:
    layer.trainable = True

long_model.compile(optimizer = optimizer_class_name, loss=loss)
K.set_value(long_model.optimizer.learning_rate,0.00001)

history = train_model(model=long_model,
    path2save=config["path2save_finetuned"],
    model_name="final_test",
    training_generator=train_generator,
    validation_generator=validation_generator,
    steps_per_epoch=nb_train_steps,
    validation_steps=nb_valid_steps,
    n_epochs=75,
    learning_rate_drop=0.5,
    learning_rate_patience=10
   )
val_loss[0,75:] = history.history['val_loss']






model = load_trained_model(main_dir+'sct_custom/data/deepseg_sc_models/t2_sc_3D.h5')
for layer in model.layers:
    layer.trainable = True


model.compile(optimizer = optimizer_class_name, loss=loss)
K.set_value(model.optimizer.learning_rate,0.00001)
print(model.summary())

history = train_model(model=model,
    path2save=config["path2save_retrained"],
    model_name="final_test",
    training_generator=train_generator,
    validation_generator=validation_generator,
    steps_per_epoch=nb_train_steps,
    validation_steps=nb_valid_steps,
    n_epochs=150,
    learning_rate_drop=0.5,
    learning_rate_patience=10
   )
val_loss[1,:] = history.history['val_loss']

np.save(main_dir+"parameters_analysis/final_test.npy",val_loss)

/home/nidebroux/data_mask/preprocess/Data_organized/iv-5/t2/iv-5_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-2/t2/iv-2_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-9/t2/pm-9_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-4/t2/pm-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-11/t2/pm-11_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-1/t2/iv-1_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-10/t2/pm-10_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-13/t2/pm-13_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-14/t2/pm-14_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/iv-4/t2/iv-4_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-6/t2/pm-6_t2_norm.nii.gz
/home/nidebroux/data_mask/preprocess/Data_organized/pm-5/t2/pm-5_t2_norm.nii.gz
/home/nidebroux/data_mask/prepro

193/193 - 71s - loss: -5.7586e-01 - val_loss: -6.4923e-01 - lr: 1.0000e-05
Epoch 3/75
193/193 - 71s - loss: -5.9859e-01 - val_loss: -6.1035e-01 - lr: 1.0000e-05
Epoch 4/75
193/193 - 71s - loss: -6.1295e-01 - val_loss: -6.2678e-01 - lr: 1.0000e-05
Epoch 5/75
193/193 - 71s - loss: -6.2253e-01 - val_loss: -6.5637e-01 - lr: 1.0000e-05
Epoch 6/75
193/193 - 71s - loss: -6.3183e-01 - val_loss: -7.0310e-01 - lr: 1.0000e-05
Epoch 7/75
193/193 - 70s - loss: -6.3241e-01 - val_loss: -7.1644e-01 - lr: 1.0000e-05
Epoch 8/75
193/193 - 72s - loss: -6.3926e-01 - val_loss: -6.7570e-01 - lr: 1.0000e-05
Epoch 9/75
193/193 - 70s - loss: -6.4339e-01 - val_loss: -7.3999e-01 - lr: 1.0000e-05
Epoch 10/75
193/193 - 72s - loss: -6.4496e-01 - val_loss: -6.8340e-01 - lr: 1.0000e-05
Epoch 11/75
193/193 - 70s - loss: -6.4989e-01 - val_loss: -7.0203e-01 - lr: 1.0000e-05
Epoch 12/75
193/193 - 72s - loss: -6.5405e-01 - val_loss: -7.3660e-01 - lr: 1.0000e-05
Epoch 13/75
193/193 - 71s - loss: -6.5656e-01 - val_loss: -7.0

Epoch 1/150
193/193 - 72s - loss: -5.0399e-01 - val_loss: -6.1022e-01 - lr: 1.0000e-05
Epoch 2/150
193/193 - 71s - loss: -5.8128e-01 - val_loss: -6.9070e-01 - lr: 1.0000e-05
Epoch 3/150
193/193 - 72s - loss: -6.0237e-01 - val_loss: -6.1512e-01 - lr: 1.0000e-05
Epoch 4/150
193/193 - 71s - loss: -6.1343e-01 - val_loss: -6.7807e-01 - lr: 1.0000e-05
Epoch 5/150
193/193 - 72s - loss: -6.2462e-01 - val_loss: -6.6328e-01 - lr: 1.0000e-05
Epoch 6/150
193/193 - 71s - loss: -6.2674e-01 - val_loss: -7.0521e-01 - lr: 1.0000e-05
Epoch 7/150
193/193 - 72s - loss: -6.3341e-01 - val_loss: -7.1542e-01 - lr: 1.0000e-05
Epoch 8/150
193/193 - 71s - loss: -6.4060e-01 - val_loss: -6.6089e-01 - lr: 1.0000e-05
Epoch 9/150
193/193 - 72s - loss: -6.4646e-01 - val_loss: -6.5774e-01 - lr: 1.0000e-05
Epoch 10/150
193/193 - 71s - loss: -6.5158e-01 - val_loss: -6.9479e-01 - lr: 1.0000e-05
Epoch 11/150
193/193 - 72s - loss: -6.5357e-01 - val_loss: -7.0622e-01 - lr: 1.0000e-05
Epoch 12/150
193/193 - 71s - loss: -6.576

Epoch 89/150
193/193 - 71s - loss: -7.0832e-01 - val_loss: -7.1085e-01 - lr: 7.8125e-08
Epoch 90/150
193/193 - 71s - loss: -7.0273e-01 - val_loss: -7.5657e-01 - lr: 7.8125e-08
Epoch 91/150
193/193 - 71s - loss: -7.0528e-01 - val_loss: -7.0905e-01 - lr: 7.8125e-08
Epoch 92/150
193/193 - 71s - loss: -7.0440e-01 - val_loss: -7.2356e-01 - lr: 7.8125e-08
Epoch 93/150
193/193 - 71s - loss: -7.0025e-01 - val_loss: -7.2803e-01 - lr: 7.8125e-08
Epoch 94/150
193/193 - 71s - loss: -7.0423e-01 - val_loss: -6.9410e-01 - lr: 7.8125e-08
Epoch 95/150
193/193 - 71s - loss: -7.0739e-01 - val_loss: -7.5056e-01 - lr: 7.8125e-08
Epoch 96/150
193/193 - 71s - loss: -7.0585e-01 - val_loss: -7.2443e-01 - lr: 7.8125e-08
Epoch 97/150
193/193 - 71s - loss: -7.0386e-01 - val_loss: -7.3391e-01 - lr: 7.8125e-08
Epoch 98/150

Epoch 00098: ReduceLROnPlateau reducing learning rate to 3.906249901319825e-08.
193/193 - 72s - loss: -7.0481e-01 - val_loss: -6.7164e-01 - lr: 7.8125e-08
Epoch 99/150
193/193 - 71s - loss: -7.0

NameError: name 'val_loss_' is not defined

In [ ]:
val_loss = np.load(main_dir+"parameters_analysis/final_test.npy")
epochs = range(5,151,5)
mean_val_loss = np.zeros((2,30))
for i in range(30):
    mean_val_loss[0,i]=np.abs(np.mean(val_loss[0,i:i+5])) 
    mean_val_loss[1,i]=np.abs(np.mean(val_loss[1,i:i+5]))

plt.plot(epochs,mean_val_loss[0,:],'o--',label = 'transfer learning')
plt.plot(epochs,mean_val_loss[1,:],'o--',label = 'no  transfer learning')

plt.title('Evolution of the validation loss with or without transfer learning ')
# show a legend on the plot
plt.ylabel('Mean of the validation loss on 5 successive epochs')
plt.xlabel('Number of epochs')
plt.legend(loc = 'lower right')
# Display a figure.
plt.show()